In [5]:
%reload_ext autoreload
%autoreload 2

from lib.basis import construct_rotation_matrix
from lib.formatting import bitstring_to_int
from lib.data_loading import load_measurements, MixedDataLoader
from lib.state_io import load_state_vector

####

import re
from pathlib import Path
from typing import Optional, Tuple, Dict, Any, Sequence, Callable

import numpy as np

import jax
import jax.lax
import jax.numpy as jnp
import optax
from flax.training.train_state import TrainState
from flax.training import checkpoints
from flax import linen as nn
from jaxtyping import PyTree

import matplotlib.pyplot as plt

data_dir = Path("data")
model_dir = "./models"
model_prefix = "rbm_amp_202506150002_0"

print(f"Data resides in                         : {data_dir}")
print(f"Amplitude RBM checkpoint to be loaded   : {model_dir}/{model_prefix}")

Data resides in                         : data
Amplitude RBM checkpoint to be loaded   : ./models/rbm_amp_202506150002_0


In [11]:
file_pattern = re.compile(r"^w_aug_ZZ.*(XX|XY).*ZZ_6400\.txt$")

file_paths = [f for f in data_dir.iterdir() if f.is_file() and file_pattern.match(f.name)]

data_bases = []
for file_path in file_paths:
    data_basis = load_measurements(file_path)
    data_bases.append(data_basis)

data = jnp.concatenate(data_bases, axis=0)

Parsing measurements: 100%|██████████| 6400/6400 [00:00<00:00, 27299.81it/s]


In [13]:
from lib.basis import construct_rotation_matrix

basis_int = jnp.array([1, 2, 3], dtype=jnp.int32)

rot = construct_rotation_matrix(basis_int)
print(rot)

TypeError: Scanned function carry input and carry output must have equal types (e.g. shapes and dtypes of arrays), but they differ:
  * the input carry component loop_carry[1] has type complex64[1,1] but the corresponding output carry component has type complex64[2,2], so the shapes do not match

Revise the scanned function so that all output types (e.g. shapes and dtypes) match the corresponding input types.

In [12]:
loader = MixedDataLoader(data, batch_size=128, drop_last=True) # we need the last batch to match the chain dimension
